# Python objects


In [3]:
from math import sqrt

def l2_norm(*dims):
    return sqrt(sum([d**2 for d in dims]))



In [120]:
from abc import ABC, abstractmethod

class BasePoint(ABC):
   
    @abstractmethod
    def magnitude(self):
        pass

In [126]:
class Point(BasePoint):
    x: int
    y: int
    _magnitude: float = None
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
      
    @property
    def magnitude(self):
        if not self._magnitude:
            self._magnitude = l2_norm(self.x,self.y)
        return self._magnitude
         
    def __repr__(self):
        return f'Point<x={self.x} y={self.y}>'
    
    def __eq__(self,other):
        return self.x==other.x and self.y == other.y
    
    def __lt__(self,other):
        return self.x < other.x

In [127]:
p = Point(1,-1)

In [128]:
p.magnitude

1.4142135623730951

In [105]:
print(p)

Point<x=1 y=-1>


In [106]:
p.__dict__

{'x': 1, 'y': -1}

In [107]:
p2 = Point(1,-1)

In [108]:
p2 == p

True

In [109]:
p2 is p

False

In [110]:
print(f'0x{id(p):x}')

0x7f633c00e160


In [111]:
print(f'0x{id(p2):x}')

0x7f633c02ab50


In [112]:
points = [Point(0,0), Point(-1,1), Point(1,1)]

In [113]:
points.sort()
points

[Point<x=-1 y=1>, Point<x=0 y=0>, Point<x=1 y=1>]

In [115]:
p.magnitude

1.4142135623730951

## Dataclasses

Dataclasses (new in 3.7) provide some "free" methods for objects that will mostly be used to hold data (state).

In [141]:
from dataclasses import dataclass, field, asdict, InitVar
from uuid import uuid4

In [143]:
def make_uuid(prefix):
    def _make_uuid():
        return prefix+'-'+str(uuid4())
    return _make_uuid

In [218]:
@dataclass(order=True)
class Point:
    x: int
    y: int 
    _magnitude: float = field(default=None, compare=False)
    _id: str = field(default_factory=make_uuid('POINT'),compare=False)


In [219]:
p = Point(1,-1)
p2 = Point(1,-1)

In [220]:
p2 == p

True

In [221]:
p

Point(x=1, y=-1, _magnitude=None, _id='POINT-7f742ad9-7e14-428b-ae96-fd72b5a47fc8')

In [222]:
p2

Point(x=1, y=-1, _magnitude=None, _id='POINT-3cb7fef8-9533-4109-bcb9-0f6cf5b72adf')

In [223]:
p2 is p

False

In [227]:
p.__dict__

{'x': 1,
 'y': -1,
 '_magnitude': None,
 '_id': 'POINT-7f742ad9-7e14-428b-ae96-fd72b5a47fc8'}

In [224]:
points = [Point(0,0), Point(-1,1), Point(1,1)]

In [225]:
points.sort()
points

[Point(x=-1, y=1, _magnitude=None, _id='POINT-3722970d-e616-470b-ba09-95b4f7b05b44'),
 Point(x=0, y=0, _magnitude=None, _id='POINT-fdbdfb2b-bb0a-41e4-a0c0-b71802afddc8'),
 Point(x=1, y=1, _magnitude=None, _id='POINT-d5d45a19-931a-4ee1-862a-f38f334338b3')]

## Marshmallow for ser-des operations

marshmallow is an ORM/ODM/framework-agnostic library for converting complex datatypes, such as objects, to and from native Python datatypes.

In [228]:
from marshmallow import Schema, fields, post_load

In [266]:
class PointSchema(Schema):
    x = fields.Integer()
    y = fields.Integer()
    
    @post_load
    def make_point(self, data, **kwargs):
        return Point(**data)

In [267]:
p = Point(1,-1)

In [271]:
PointSchema().load({
    'x': '1',
    'y':'2'
})

Point(x=1, y=2, _magnitude=None, _id='POINT-84b85a48-ae01-44f9-bfaa-bc8d6934bbed')

In [272]:
PointSchema().dump(p)

{'x': 1, 'y': -1}

In [258]:
schema = PointSchema(only=['y'], many=True)

In [259]:
schema.dump([p,p,p])

[{'y': -1}, {'y': -1}, {'y': -1}]

### Marshal to dictionaries and lists of dictionaries with automatic validation (from strings)

In [273]:
PointSchema().dump(p)

{'x': 1, 'y': -1}

In [274]:
PointSchema().load({
    'x':'a',
    'y':-1,
    'z': 10
})

ValidationError: {'x': ['Not a valid integer.'], 'z': ['Unknown field.']}

In [278]:
PointSchema(many=True).load([
    {'x':1, 'y':10},
    {'x':0, 'y':0}
])

[Point(x=1, y=10, _magnitude=None, _id='POINT-6d1122f8-6ddc-46e1-91b6-d34747aa9551'),
 Point(x=0, y=0, _magnitude=None, _id='POINT-c4c67432-f7ba-4328-abb5-ea6637e6765e')]